## **N^2 - 1 Puzzle Problem**
Write a Python/ Java/ C/ C++ program to solve the (n^2 − 1)-puzzle problem using the following search algorithms:
- Breadth-First Search (BFS)
- Depth-First Search (DFS)
- Iterative Deepening (ID)

### 1. Breadth-First Search (BFS)

In [3]:
import numpy as np
from itertools import permutations
import random

n = 4

def generate_puzzle_numbers(n=3):
    numbers = []
    digits = list(range(n*n))
    
    for _ in range(20):
        current = digits.copy()
        for i in range(len(current)-1, 0, -1):
            j = np.random.randint(0, i+1)
            current[i], current[j] = current[j], current[i]
        
        num = int(''.join(map(str, current)))
        numbers.append(num)
    
    return numbers

numbers = generate_puzzle_numbers()
start_states = numbers[:10]
goal_states = numbers[10:]
print("Start states:")
print(start_states)
print("Goal states:")
print(goal_states)
print()

for start, goal in zip(start_states, goal_states):
    start = str(start)
    goal = str(goal)
    if len(start) < (n * n):
        start = '0' + start
    if len(goal) < (n * n):
        goal = '0' + goal
    print(f"Start: {start}, Goal: {goal}")
    print()
    
    break
    

Start states:
[27684153, 820436571, 402563718, 564281073, 573168204, 53421876, 874302516, 583026417, 246378150, 350164278]
Goal states:
[812546307, 180746352, 308125647, 237840561, 215637084, 865437102, 276108354, 305712864, 321684507, 354187206]

Start: 027684153, Goal: 0812546307

